In [1]:
# !pip install pandas==1.3
# !pip install tqdm==4.61.2
# ! pip install strbalance
import os
import pandas as pd
from tqdm import tqdm, notebook
notebook.tqdm().pandas()
%load_ext autoreload

import twee_utils
import importlib
import tqdm, pandas
importlib.reload(twee_utils)
importlib.reload(tqdm)
importlib.reload(pandas)

0it [00:00, ?it/s]

<module 'pandas' from '/Users/alex/anaconda3/envs/spark/lib/python3.7/site-packages/pandas/__init__.py'>

In [ ]:
# !which pip
# /Users/alex/anaconda3/envs/spark/bin/pip
# %pip install strbalance

## Get Twine Files


In [2]:
downloads = "../game_downloads/"
game_html = "../game_html/"

In [3]:
twee_utils.try_unzip(downloads + 'breengrub.zip', game_html + 'temp')

'../game_html/temp'

In [ ]:
import twee_utils
import importlib
import tqdm, pandas
importlib.reload(twee_utils)
importlib.reload(tqdm)
importlib.reload(pandas)

# Test that we can get a zip, a folder, and an html file this way
html1 = twee_utils.get_html_source(downloads + 'breengrub.zip')
html2 = twee_utils.get_html_source(downloads + 'Iron Kid Interactive Story/')
html3 = twee_utils.get_html_source(downloads + '3_Garde_Final.html')
print(html2)

In [ ]:
!ls ../game_downloads/

In [ ]:
%autoreload 2
import twee_utils
import importlib
importlib.reload(twee_utils)

twine = pd.DataFrame({'file': os.listdir(downloads)})
twine['path'] = twine.file.apply(lambda x: os.path.join(downloads, x))
twine['filetype'] = twine.file.apply(lambda x: x.split('.')[-1] if '.' in x else 'unk')
twine = twine[~twine.filetype.isin(('exe', 'quest', 'apk'))]
twine['html'] = twine.path.progress_apply(lambda x: twee_utils.get_html_source(x))

In [ ]:
twine.to_pickle('twine.pkl')
twine.head(30)

## Load saved twine data

In [4]:
import pandas as pd
twine = pd.read_pickle('twine.pkl')

In [ ]:
valid_twine = twine[twine.html != '']
valid_twine = valid_twine.reset_index()
valid_twine

In [ ]:
valid_twine['is_untweeable'] = valid_twine.html.apply(lambda x: twee_utils.is_untweeable(x))
valid_twine
valid_twine[valid_twine.is_untweeable]
tweeable_twine = valid_twine[valid_twine.is_untweeable]
tweeable_twine

In [ ]:

importlib.reload(twee_utils)
def untwee(s):
    html = s['html']
    file = twee_utils.make_temp_file(html)
    twee, error = twee_utils.untwee(file)
    twee_utils.delete_temp_file(file)
    return twee, error

# head = valid_twine.head(5)

tweeable_twine[['twee', 'twee_error']] = tweeable_twine.progress_apply(untwee, axis=1, result_type='expand')
tweeable_twine
# head['twee'], head['twee_error'] = valid_twine.html.apply(
#     (lambda x: untwee(x)), result_type='expand'
# ).T.values

In [ ]:
tweeable_twine.twee = tweeable_twine.twee.str.decode("utf-8")
tweeable_twine.to_pickle('twee.pkl')

## Read twee pickle

In [13]:
tweeable_twine = pd.read_pickle('twee.pkl')

## TODO : Fix is_valid_twee
So that we can get process more games

In [14]:
tweeable_twine['valid_twee'] = tweeable_twine.twee.progress_apply(twee_utils.is_valid_twee)
tweeable_twine.to_pickle('valid_twee.pkl')
tweeable_twine

  0%|          | 0/82 [00:00<?, ?it/s]

,index,file,path,filetype,html,is_untweeable,twee,twee_error,valid_twee
0,2,GraySkiesAlpha.html,../game_downloads/GraySkiesAlpha.html,html,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,":: Debug Silo\n<<set $detkey to ""true"">>\n\n<<...",None,False
4,12,breengrub.zip,../game_downloads/breengrub.zip,zip,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,":: pour out onto the floor.\nYou <<replace ""dr...",None,False
7,17,TheSaintsofHorsesQuestfortheTraitorSaint.html,../game_downloads/TheSaintsofHorsesQuestforthe...,html,<!DOCTYPE html>\n<!--[if lte IE 8]> <html clas...,True,:: saint mercury [Twine.image]\ndata:image/jpg...,None,False
8,19,A_Month_in_Z32-LDJAM32.zip,../game_downloads/A_Month_in_Z32-LDJAM32.zip,zip,<!DOCTYPE html>\n<!--[if lte IE 8]> <html clas...,True,:: Mountain [coreLoop explore]\n<<set $mountai...,None,False
9,20,The Domovoi,../game_downloads/The Domovoi,unk,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: Start2 [start]\n<<stopallsound>>\n<<fadeins...,None,False
...,...,...,...,...,...,...,...,...,...
245,449,Cabin Fever Complete,../game_downloads/Cabin Fever Complete,unk,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: fireplace3\n<<set $awareness += 2>><<set $f...,None,False
250,460,Lights Out Please.html,../game_downloads/Lights Out Please.html,html,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: But why should I have to suffer alone? [man...,None,False
255,471,rocketjump.html,../game_downloads/rocketjump.html,html,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",True,:: Ralph Lauren\nYou tweet your succcess story...,None,False
266,501,colors of sound.html,../game_downloads/colors of sound.html,html,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: love\nplease tell me i still matter and <<c...,None,False


In [15]:
tweeable_twine = pd.read_pickle('valid_twee.pkl')

## Look at removing all twee games without macros
This isn't valid, since there aren't enough twee stories

Instead, we create an 'absent_macros' column.
### TODO need to remove other things like stylesheets

In [16]:
tweeable_twine['macros'] = tweeable_twine.twee.progress_apply(twee_utils.get_macros)
tweeable_twine['absent_macros'] = tweeable_twine.twee.progress_apply(twee_utils.replace_macros)
tweeable_twine

full_with_macros = tweeable_twine[tweeable_twine.macros.apply(len) > 0]
print('with macros', full_with_macros.shape)
full_without_macros = tweeable_twine[tweeable_twine.macros.apply(len) == 0]
print('without macros', full_without_macros.shape)
full_without_macros.head(1)

full_with_macros[['twee', 'macros', 'absent_macros']].sample(10)

# Reset the index, make sure its on the full twee story level now
tweeable_twine = tweeable_twine.reset_index(drop=True, col_level=1)
tweeable_twine = tweeable_twine.reset_index(drop=True)
tweeable_twine

  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

with macros (64, 11)
without macros (18, 11)


,index,file,path,filetype,html,is_untweeable,twee,twee_error,valid_twee,macros,absent_macros
0,2,GraySkiesAlpha.html,../game_downloads/GraySkiesAlpha.html,html,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,":: Debug Silo\n<<set $detkey to ""true"">>\n\n<<...",None,False,"[set $detkey to ""true"", set $poweron to ""true""...",:: Debug Silo\n\n\n\n\n[[Test|silos]]\n\n\n:: ...
1,12,breengrub.zip,../game_downloads/breengrub.zip,zip,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,":: pour out onto the floor.\nYou <<replace ""dr...",None,False,"[replace ""drop the gun."", replace ""dry-retch.""...",:: pour out onto the floor.\nYou drop the gun....
2,17,TheSaintsofHorsesQuestfortheTraitorSaint.html,../game_downloads/TheSaintsofHorsesQuestforthe...,html,<!DOCTYPE html>\n<!--[if lte IE 8]> <html clas...,True,:: saint mercury [Twine.image]\ndata:image/jpg...,None,False,"[if visited() > 20, endif, if visited(""SaintsP...",:: saint mercury [Twine.image]\ndata:image/jpg...
3,19,A_Month_in_Z32-LDJAM32.zip,../game_downloads/A_Month_in_Z32-LDJAM32.zip,zip,<!DOCTYPE html>\n<!--[if lte IE 8]> <html clas...,True,:: Mountain [coreLoop explore]\n<<set $mountai...,None,False,"[set $mountainVisit += 1, set $findChance = Ma...",:: Mountain [coreLoop explore]\n\n\n\\nYou loo...
4,20,The Domovoi,../game_downloads/The Domovoi,unk,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: Start2 [start]\n<<stopallsound>>\n<<fadeins...,None,False,"[stopallsound, fadeinsound $intro, nobr, set $...",:: Start2 [start]\n\n\n\n\n\n\n//Recorded in /...
...,...,...,...,...,...,...,...,...,...,...,...
77,449,Cabin Fever Complete,../game_downloads/Cabin Fever Complete,unk,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: fireplace3\n<<set $awareness += 2>><<set $f...,None,False,"[set $awareness += 2, set $fireplace = 1, set ...",:: fireplace3\n\\nYou look inside the fireplac...
78,460,Lights Out Please.html,../game_downloads/Lights Out Please.html,html,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: But why should I have to suffer alone? [man...,None,False,"[timedgoto ""Victorian"" 5s, timedgoto ""i claw"" ...",:: But why should I have to suffer alone? [man...
79,471,rocketjump.html,../game_downloads/rocketjump.html,html,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",True,:: Ralph Lauren\nYou tweet your succcess story...,None,False,"[set $stability = $stability - 10 , display 'S...",:: Ralph Lauren\nYou tweet your succcess story...
80,501,colors of sound.html,../game_downloads/colors of sound.html,html,﻿<!DOCTYPE html>\n<!--[if lte IE 8]><html clas...,True,:: love\nplease tell me i still matter and <<c...,None,False,"[continue""you still love me"", endcontinue, tim...",:: love\nplease tell me i still matter and you...


## Get passages the new way and create context

In [ ]:
import analysis
importlib.reload(contextual_tree)
importlib.reload(analysis)
import contextual_tree
from contextual_tree import ContextualTweeTree
importlib.reload(twee_utils)
test = tweeable_twine.head(2)
nodes = test.twee.progress_apply(lambda x: ContextualTweeTree.create(twee=x))

# passages = tweeable_twine.twee.apply(lambda x: [(x, for x in ContextualTweeTree.create(twee=x)]).explode()


In [ ]:
importlib.reload(twee_utils)
passages = test.twee.apply(lambda x: [x for x in twee_utils.split_passages(x)]).explode()

## Get Passages (the old fashioned way)

In [17]:
# Get passages from the 'absent_macros' column created earlier
importlib.reload(twee_utils)
passages = tweeable_twine.twee.apply(lambda x: [x for x in twee_utils.split_passages(x)]).explode()

In [18]:
pd.set_option('max_colwidth',200)
print(passages.size)
passages[passages.index==81]
# passages.groupby(passages.index).get_group(1)

10784


81    ::walk\n9 kilometres.\n\nAway from the more sparsely populated area.\n\nInto the urban center.\n\nShops, sights, people, observations.\n\nMalls, shops, bars, pubs, stores, restaurants, services.\n...
81    ::Start\nThis is a ... kind of a sequel to my previous twine game, <a href="http://limbclock.itch.io/the-process">The Process</a>\n\nHowever, the entity in this game and the previous one may or ma...
81    ::computer\nThere's nothing to do inside but to be in front of the computer.\n\nTo lose yourself in the rabbit hole of <a href="http://www.tvtropes.com">TVTropes</a> or Wikipedia.\n\nTo waste in f...
81          ::gamestart\n<a href="http://limbclock.itch.io/the-process">The process</a> is always there to help you.\n\nLike a...kickstart for the creative process.\n\nNow, however it's time to [[write|1]]
81    ::piece you wrote\nWhat piece was it anyway?\n\nHow should i know. I gave you options. In your mind you, the player decided, "Yes, this is the piece that i would write. O

In [19]:
passages.to_pickle('passages.pkl')
passages

0                                                                                                                     ::Debug Silo\n<<set $detkey to "true">>\n\n<<set $poweron to "true">>\n\n[[Test|silos]]
0     ::officeupstairs\nHalf of the building has collapsed. To your left is a large locker room, with numbers ranging from 100 to 250\n\nOpen locker number <<textinput $lockernum [[Check locker|officelo...
0     ::redsfinal\n<<if $healedrads is "false">>\nThe radiation sickness has taken to much of a toll on your body. You collapse as your organs shutdown. And you were so close. \n\n<<else>>\nYou head up ...
0     ::SouthA1\nYou walk for several miles before reaching the mountains. You could try and go through them, but it would be at least a weeks trip through them. You'd die before you made it across. \n\...
0     ::opheimletin\n"Name and Purpose! Tell or leave."\n\n[["My name is <<print $PlayerName>> I'm just passing through on my way north"|opheimpass]]\n\n[["Names LetMeIn and I'

# Process and clean passages

In [31]:
passages = pd.read_pickle('passages.pkl')
passages = pd.DataFrame(passages)
passages

,twee
0,"::Debug Silo\n<<set $detkey to ""true"">>\n\n<<set $poweron to ""true"">>\n\n[[Test|silos]]"
0,"::officeupstairs\nHalf of the building has collapsed. To your left is a large locker room, with numbers ranging from 100 to 250\n\nOpen locker number <<textinput $lockernum [[Check locker|officelo..."
0,"::redsfinal\n<<if $healedrads is ""false"">>\nThe radiation sickness has taken to much of a toll on your body. You collapse as your organs shutdown. And you were so close. \n\n<<else>>\nYou head up ..."
0,"::SouthA1\nYou walk for several miles before reaching the mountains. You could try and go through them, but it would be at least a weeks trip through them. You'd die before you made it across. \n\..."
0,"::opheimletin\n""Name and Purpose! Tell or leave.""\n\n[[""My name is <<print $PlayerName>> I'm just passing through on my way north""|opheimpass]]\n\n[[""Names LetMeIn and I'm here to beat up curious ..."
...,...
81,::postscript\nThis twine was written in an half an hour or so.\n\nI'd like to thank the following twitter users:\n\n@richardgoodness for helping me getting back into writing and giving me encourag...
81,::impatient\nInternet is wrecking your attention span.\n\nWhy am i still writing this?\n\nI wanna do something else.\n\nTWEETS WAITING\n\nSTATUSSES TO LIKE.\n\nWhat have we all become? We're all s...
81,"::opinion\nThere are many things to be opinionated about:\n\nGender issues\nVideo games\nPolitics\nNews\nSocial media, heck even how to pay you phone bill is something to [[write|writing]] about."
81,::software\nWith each update.\n\nThe support for older files disappears.\n\nSometimes they keep a copy of the older version of the software on some godforsaken old model computer just so that the ...


In [32]:
importlib.reload(twee_utils)
max_length = 2048

# Rename twee -> passage
if 'twee' in passages:
    passages['passage'] = passages['twee']
    del passages['twee']
print("Initial shape", passages.shape)

passages['original_passage'] = passages.passage

passages['char_count'] = passages.passage.apply(len)

# Cut out the long passages and useless passages
passages = passages[
    ~passages.passage.apply(lambda x: twee_utils.is_special_passage(x) or twee_utils.is_empty_passage(x))
]
passages.passage = passages.passage.apply(lambda x: x[:max_length])
print("Shape after first clean ", passages.shape)

# Clean the twee - for some reason this takes a long time
# Remove images, duplicate newlines, etc
print("Removing and saving macros")
passages['macros'] = passages.passage.progress_apply(twee_utils.get_macros)
passages['passage'] = passages.passage.progress_apply(twee_utils.replace_obvious_macros)
passages['passage'] = passages.passage.progress_apply(twee_utils.replace_macros)
print("Removing images and such") 
passages['passage'] = passages.passage.progress_apply(twee_utils.clean_twee)
print('Shape after cleaning', passages.shape)

# Split into lines
passages['lines'] = passages.passage.progress_apply(twee_utils.split_lines)
passages['line_count'] = passages.lines.progress_apply(len)
passages['title'] = passages.lines.progress_apply(twee_utils.get_title)
if 'lines' in passages:
    del passages['lines']

# Drop Duplicates
passages.drop_duplicates(subset=['passage'])
print()
print("Final shape", passages.shape)

Initial shape (10784, 1)
Shape after first clean  (10332, 3)
Removing and saving macros


/Users/alex/anaconda3/envs/spark/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]

...

  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]

  0%|          | 0/10332 [00:00<?, ?it/s]


Final shape (10332, 6)


,passage,original_passage,char_count,macros,line_count,title
0,::Debug Silo\n[[Test|silos]],"::Debug Silo\n<<set $detkey to ""true"">>\n\n<<set $poweron to ""true"">>\n\n[[Test|silos]]",82,"[set $detkey to ""true"", set $poweron to ""true""]",2,::Debug Silo
0,"::officeupstairs\nHalf of the building has collapsed. To your left is a large locker room, with numbers ranging from 100 to 250\nOpen locker number \n[[Return to Lobby|officearmy]]","::officeupstairs\nHalf of the building has collapsed. To your left is a large locker room, with numbers ranging from 100 to 250\n\nOpen locker number <<textinput $lockernum [[Check locker|officelo...",233,[textinput $lockernum [[Check locker|officelocker]]],4,::officeupstairs
0,::redsfinal\nThe radiation sickness has taken to much of a toll on your body. You collapse as your organs shutdown. And you were so close. \nYou head up a staircase into Reds base. You reach a sma...,"::redsfinal\n<<if $healedrads is ""false"">>\nThe radiation sickness has taken to much of a toll on your body. You collapse as your organs shutdown. And you were so close. \n\n<<else>>\nYou head up ...",304,"[if $healedrads is ""false"", else, endif]",4,::redsfinal
0,"::SouthA1\nYou walk for several miles before reaching the mountains. You could try and go through them, but it would be at least a weeks trip through them. You'd die before you made it across. \n[...","::SouthA1\nYou walk for several miles before reaching the mountains. You could try and go through them, but it would be at least a weeks trip through them. You'd die before you made it across. \n\...",256,[],4,::SouthA1
0,"::opheimletin\n""Name and Purpose! Tell or leave.""\n[[""My name is I'm just passing through on my way north""|opheimpass]]\n[[""Names LetMeIn and I'm here to beat up curious rat faced assholes""|opheim...","::opheimletin\n""Name and Purpose! Tell or leave.""\n\n[[""My name is <<print $PlayerName>> I'm just passing through on my way north""|opheimpass]]\n\n[[""Names LetMeIn and I'm here to beat up curious ...",268,[print $PlayerName],6,::opheimletin
0,"::Character\n""Ok everything seems to be in order. So , you have strength, speed, and smarts.""\n""I've printed out the results, if you want to look over it again""\n[[Look at character sheet|characte...","::Character\n""Ok everything seems to be in order. So <<print $PlayerName>>, you have <<print $strength>> strength, <<print $speed>> speed, and <<print $smarts>> smarts.""\n\n""I've printed out the r...",321,"[print $PlayerName, print $strength, print $speed, print $smarts]",5,::Character
0,::launchfail\nAccess Denied! Both Detenation Keys Required to Authenticate Launch.,::launchfail\nAccess Denied! Both Detenation Keys Required to Authenticate Launch.\n\n<<return>>,93,[return],2,::launchfail
0,::NorthA1\nYou arrive at a small rocky out cropping. Some large and menacing looking birds circle above you. There doesn't seem to be anything else around.\n[[Go North|NorthA2]]\n[[Go to Red's|red...,::NorthA1\nYou arrive at a small rocky out cropping. Some large and menacing looking birds circle above you. There doesn't seem to be anything else around.\n\n[[Go North|NorthA2]]\n\n[[Go to Red's...,249,[],6,::NorthA1
0,::teenattack\n\\nYou dive at the teens but they dog pile you. The arm is destroyed in the scuffle and you are bruised all over.\n[[Exit Alley (with a bruised pride)|opheimsquare]]\n\\nYou dive at ...,::teenattack\n<<if $strength is 1 >>\\nYou dive at the teens but they dog pile you. The arm is destroyed in the scuffle and you are bruised all over.\n\n[[Exit Alley (with a bruised pride)|opheims...,381,"[if $strength is 1 , elseif $strength > 1 , set $jimarm to 1, endif]",7,::teenattack
0,::NorthA2\nYou reach a stretch of highway filled with cars. It looks like these people were stuck in traffic when the bombs hit. To the north you can see a makeshift shelter made of wrecked cars. ...,::NorthA2\nYou reach a stretch of highway filled with cars. It looks like these people were stuck in traf

In [43]:
pd.set_option('display.max_rows', 300)
passages[passages.index==5]

,passage,original_passage,char_count,macros,line_count,title
5,"::caused\n""I thought you were a good friend and it turns out you have a mental problem too, narcissism. You have no empathy, you only hang out with people that idolize you. It's about FRIENDSHIP. ...","::caused\n""I thought you were a good friend and it turns out you have a mental problem too, narcissism. You have no empathy, you only hang out with people that idolize you. It's about FRIENDSHIP. ...",392,[],2,::caused
5,::form\nwhy play games?\nyour breathing\nis half the answer\nbut like the rest\nyou dress\nto [[sleep]],::form\nwhy play games?\n\nyour breathing\nis half the answer\nbut like the rest\nyou dress\nto [[sleep]],98,[],7,::form
5,"::dead\nfleshy, stiff\ngrey grains\nof sooty dread\nwould nurse the tears from my fragile facial [[veins]]","::dead\nfleshy, stiff\ngrey grains\nof sooty dread\nwould nurse the tears from my fragile facial [[veins]]",102,[],5,::dead
5,"::thank you.\n""Don't take this with a grain of salt. For your sake, move to ┼╛σ╛±┼╛, spiralling down into a foreseeable realization that you are not special, and that there is no quality you harbo...","::thank you.\n""Don't take this with a grain of salt. For your sake, move to ┼╛σ╛±┼╛, spiralling down into a foreseeable realization that you are not special, and that there is no quality you harbo...",252,[],3,::thank you.
5,"::sorrows [sorrows]\n""I hope you're infertile. You disgust me. I'm ashamed of exchanging saliva with someone so wretched.""\n[[<<Back|≡]]","::sorrows [sorrows]\n""I hope you're infertile. You disgust me. I'm ashamed of exchanging saliva with someone so wretched.""\n\n[[<<Back|≡]]",135,[],3,::sorrows [sorrows]
5,::Start\ncontent warning: abusive language\n[[play video|video]],::Start\ncontent warning: abusive language\n[[play video|video]],62,[],3,::Start
5,"::lips\nall that the ears hear, or the nose tastes\nbecome the grains of the visionary, \nthe painful [[unmouthings]] of the prophet...","::lips\nall that the ears hear, or the nose tastes\nbecome the grains of the visionary, \nthe painful [[unmouthings]] of the prophet...",132,[],4,::lips
5,::video\nTHIS BEDROOM \nIS A BLAND EXCUSE\nFOR A [[WOMB]],::video\n<<playvideo zesjptihInc 0 loop sound>>\n\nTHIS BEDROOM \nIS A BLAND EXCUSE\nFOR A [[WOMB]],94,[playvideo zesjptihInc 0 loop sound],4,::video
5,::victim\nand misshhaped\n[[asking]],::victim\nand misshhaped\n[[asking]],34,[],3,::victim
5,"::mirror\n★ \n""I still love you- unconditionally. Intense, exploitative but intimate encounters that have since then ended have not shed a single tear from me over the loss of whomever but have ra...","::mirror\n★ \n""I still love you- unconditionally. Intense, exploitative but intimate encounters that have since then ended have not shed a single tear from me over the loss of whomever but have ra...",518,[],3,::mirror


In [44]:
pd.set_option('display.max_rows', 20)

In [46]:
import analysis, contextual_tree
importlib.reload(analysis)
importlib.reload(contextual_tree)
from contextual_tree import ContextualTweeTree
pd.set_option('display.max_rows', 20)
first = list(passages[passages.index == 5].passage)
print(first)
tree, passage_dict = ContextualTweeTree.create(passages=list(first))

['::caused\n"I thought you were a good friend and it turns out you have a mental problem too, narcissism. You have no empathy, you only hang out with people that idolize you. It\'s about FRIENDSHIP. NOT LEVERAGE. IT\'S ABOUT DOING THINGS THAT THE OTHER WOULD DO FOR YOU. you are fucking [[toxic]]. why don\'t you care how i feel. WHY CAN\'T YOU FUCKING FIND IT IN YOURSELF TO BE CONCERNED ABOUT ME?"', '::form\nwhy play games?\nyour breathing\nis half the answer\nbut like the rest\nyou dress\nto [[sleep]]', '::dead\nfleshy, stiff\ngrey grains\nof sooty dread\nwould nurse the tears from my fragile facial [[veins]]', '::thank you.\n"Don\'t take this with a grain of salt. For your sake, move to ┼╛σ╛±┼╛, spiralling down into a foreseeable realization that you are not special, and that there is no quality you harbor that could ever merit what you did to me."\n[[<<Back|≡]]', '::sorrows [sorrows]\n"I hope you\'re infertile. You disgust me. I\'m ashamed of exchanging saliva with someone so wretche

KeyboardInterrupt: 

In [ ]:
tree.render_root()

In [ ]:
from anytree.search import findall
title = list(passages[passages.index == 5].title)[3]
# title = twee_utils.title_to_text(title).lower()
title = ':: strength1'
print(title)
node = findall(tree, lambda node: title == node.name.lower())
print([a.title for a in node])

In [ ]:
# Check if each passage is valid
passages['valid'] = passages.passage.progress_apply(twee_utils.is_valid_passage)
passages

In [ ]:
# pd.set_option("max_rows", 250)
num_invalid = passages[~passages.valid].shape[0]
print('invalid', passages[~passages.valid].shape)
passages[~passages.valid].sample(50)
assert(num_invalid < 400)
print('valid', passages[passages.valid].shape)

In [ ]:
# Prepare passages for generation
importlib.reload(twee_utils)
valid_passages = passages[passages.valid]
pd.set_option('max_colwidth', 200)
# valid_passages['gen_input'] = valid_passages.passage.apply(twee_utils.twee_to_gen_format)
valid_passages[['prompt', 'completion']] = valid_passages.apply(lambda x: twee_utils.twee_to_gen_format_3(x.passage), axis=1, result_type="expand")
valid_passages

## Export to a file

In [ ]:
# with open('twee_training.txt', 'w') as file:
#     file.writelines(passages.gen_input)

name = 'twee_training_1'
    
if 'prompt' in valid_passages and 'completion' in valid_passages:
    pd.set_option('max_colwidth', 500)
#     gpt_json = valid_passages[['prompt', 'completion']]
    
    with open(f'{name}.json', 'w') as file:
        lines = []
        for i, line in valid_passages.iterrows():
            lines.append(line[['prompt', 'completion']].to_json(orient='columns') + '\n')
        file.writelines(lines)
        
    valid_passages.to_pickle(f'{name}.pkl')
        
print('wrote')

testing = pd.DataFrame(valid_passages.prompt)
testing['title'] = testing.prompt.progress_apply(lambda x: twee_utils.gen_to_twee_format_3(x))
testing['text'] = testing.title.progress_apply(lambda x: twee_utils.title_to_text(x))
testing['text'].to_csv(f'{name}_titles.csv')